In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [14]:
net = torch.jit.load('resnet18_pipe_classification_400imgs_acc85_jit_script.pt', map_location='cpu')
net.eval()

RecursiveScriptModule(
  original_name=ResNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
  (layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)

In [12]:
test_transforms = transforms.Compose(
[
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [47]:
import cv2
import numpy as np

cap = cv2.VideoCapture(2)

if (cap.isOpened()== False):
    print("Error opening video file")

while(cap.isOpened()):

   
    ret, frame = cap.read()
    if ret == True:
  
        #cv2.imshow('Frame', frame)
        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        inputs = torch.unsqueeze(test_transforms(pil_img), 0) #unsqueeze convert 3 dim to 4 dim
        outputs = net(inputs)
        _, preds = torch.max(outputs.data, 1)
        
        
        if preds == 0:
            cv2.putText(frame, "non pipe", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0, 0, 255), 2, cv2.LINE_AA) 
        else:
            cv2.putText(frame, "pipe", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0, 255, 0), 2, cv2.LINE_AA)
            
          
        
        cv2.imshow('Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
 
    else:
        break


cap.release()
cv2.destroyAllWindows()
